In [2]:
from data_tools.api import *
from utilscht.Data import *
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from joblib import Parallel,delayed
import tushare as ts
import datetime
import math
%config InlineBackend.figure_format ='retina'
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


START_YEAR=2014

DB_INFO = dict(host='192.168.1.234',
               user='winduser',
               password='1qaz@WSX',
               db='wind')

conn = pymysql.connect(**DB_INFO, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

In [ ]:
def apply_parallel(df_grouped, func, n_jobs=16, backend='loky', as_index=True, **kwargs):
    """
    This is for parallel between grouped generated by pd.DataFrame.groupby
    :param df_grouped:
    :param func:
    :param n_jobs:
    :param backend:
    :param kwargs:
    :return:
    """

    names = []
    groups = []
    for name, group in df_grouped:
        names.append(name)
        groups.append(group)

    results = Parallel(n_jobs=n_jobs, verbose=5, backend=backend, batch_size='auto') \
        (delayed(func)(group, **kwargs) for group in groups)

    return pd.concat(results, keys=names if as_index else None)

1. 
+ 计算上一年所有研报对已发布年报的偏差
+ 将偏差对市值、行业、距离年末时间进行回归
+ 取回归残差作为分析师自身偏差

2. 
+ 对过去3个月（过去三个月研报数小于5的个股使用过去6月的数据，对于过去六个月没有研报的个股填充nan值）
 的数据进行过滤，同一分析师对同一个股的多篇研报只取最后一篇

3. 
+ 对过去三个月的一致预期数据按照时间和分析师偏差进行双重加权（时间越近权重越大，分析师偏差越小权重越大）

4. 
+ 如果企业发布了业绩预报或者快报，则直接采取业绩预报或者快报的数据

5.                  
+ 附：滚动指标计算（向未来一年进行滚动）

In [ ]:
df_tot_share[df_tot_share["sid"]=='600380.SH']

In [ ]:
df_FA_EPS[df_FA_EPS["sid"]=='600380.SH'].sort_values("REPORTING_PERIOD")

In [ ]:
temp_df = df_est[(df_est["sid"]=='000761.SZ') & (df_est["REPORTING_PERIOD"] == '20211231')]
temp_df=temp_df[temp_df["DataDate"]>'20181110']
df = Process_Single_StkPeriod(temp_df)
plt.plot(pd.to_datetime(df.index),np.array(df["con_est_eps"]))

In [ ]:
df

In [ ]:
temp_df = df_est[(df_est["sid"]=='000016.SZ') & (df_est["REPORTING_PERIOD"] == '20191231')]
temp_df=temp_df[temp_df["DataDate"]>'20181118']
temp_df[["sid","DataDate","EST_EPS_DILUTED","analyst_avg_bias"]].iloc[0:40]

In [ ]:
##计算上一年所有研报对已发布年报的偏差

#获取行业和市值
indus_size_df=query_table("DailyBar",start_date="{}0101".format(START_YEAR),end_date="20201231",fields=["L1_INDUSTRY","mktcap"])
indus_size_df["DataDate"]=indus_size_df["DataDate"].apply(lambda x:str(x))

#获取研报预期数据
sql = """SELECT S_INFO_WINDCODE, EST_DT, REPORTING_PERIOD,RESEARCH_INST_NAME, ANALYST_NAME,EST_NET_PROFIT 
from ASHAREEARNINGEST where EST_DT between '{}0101' and '20201231'""".format(START_YEAR)   
df_est = pd.read_sql_query(sql, conn)
df_est.rename(columns={"S_INFO_WINDCODE":"sid","EST_DT":"DataDate"},inplace=True)
df_est=df_est.sort_values(["sid","DataDate","REPORTING_PERIOD"])
df_est=pd.merge(df_est,indus_size_df,on=["sid","DataDate"],how="left")

#获取发布的年报数据
sql = """SELECT S_INFO_WINDCODE, ANN_DT, REPORT_PERIOD,S_FA_DEDUCTEDPROFIT,S_FA_EXTRAORDINARY
from ASHAREFINANCIALINDICATOR where ANN_DT between '{}0101' and '20201231'""".format(START_YEAR)
df_FA_EPS = pd.read_sql_query(sql, conn)
df_FA_EPS.rename(columns={"S_INFO_WINDCODE":"sid","REPORT_PERIOD":"REPORTING_PERIOD"},inplace=True)
df_est=pd.merge(df_est,df_FA_EPS,on=["sid","REPORTING_PERIOD"],how="left")

#获取总股本信息
sql="select S_INFO_WINDCODE,TOT_SHR,CHANGE_DT from AShareCapitalization".upper()
df_tot_share=pd.read_sql_query(sql,conn)
df_tot_share.rename(columns={"S_INFO_WINDCODE":"sid"},inplace=True)
df_tot_share=df_tot_share.sort_values(["sid","CHANGE_DT"])
df_now_tot_share=df_tot_share.groupby('sid',as_index=False).apply(lambda x:x.iloc[-1])

#计算前复权每股预期收益
df_est=pd.merge(df_est,df_now_tot_share,on='sid',how='left')
df_est['EST_EPS_DILUTED']=df_est['EST_NET_PROFIT']/df_est['TOT_SHR']
df_est['S_FA_EPS_DILUTED']=(df_est['S_FA_DEDUCTEDPROFIT']+df_est['S_FA_EXTRAORDINARY'])/df_est['TOT_SHR']/10000

In [ ]:
#计算距离年末的月数（第二年初的预测月数为负值）
df_1y_est=df_est.groupby(["sid","DataDate","ANALYST_NAME"],as_index=False).apply(lambda x:x.iloc[0]).reset_index(drop=True)
df_1y_est["Date_to_Reporting"]=df_1y_est[["DataDate","REPORTING_PERIOD"]].apply(lambda x:int(x[1][4:6])-int(x[0][4:6]) if x[0][0:4]==x[1][0:4] else int(x[1][4:6])-int(x[0][4:6])-12,axis=1)

#计算偏差比例
df_1y_est["Analyst_Bias"]=np.abs(df_1y_est["S_FA_EPS_DILUTED"]-df_1y_est["EST_EPS_DILUTED"])/df_1y_est["S_FA_EPS_DILUTED"]

#按照年度将偏差对行业、市值、距离年末月数进行回归取残差
def Regress_by_Year(df):
    df=df[pd.notnull(df["Analyst_Bias"])&pd.notnull(df["L1_INDUSTRY"])]
    if len(df)==0:
        return df
    bias=np.array(df["Analyst_Bias"])
    
    indus_dummy=np.array(pd.get_dummies(df["L1_INDUSTRY"]))
    mktv=np.array(df["mktcap"])
    date_to_reporting=np.array(df["Date_to_Reporting"])
    
    model=sm.OLS(bias,np.column_stack([indus_dummy,mktv,date_to_reporting]))
    result=model.fit()
    df["adj_analyst_bias"]=result.resid
    
    return df

df_1y_est["adj_analyst_bias"]=np.nan
df_1y_est=df_1y_est.groupby("REPORTING_PERIOD",as_index=False).apply(lambda x:Regress_by_Year(x)).reset_index(drop=True)

#按照预测年度、分析师来计算偏差平均值作为下一年度该分析师的可信水平
analyst_avg_bias=df_1y_est.groupby(["REPORTING_PERIOD","ANALYST_NAME"]).apply(lambda x:np.nan if len(x["adj_analyst_bias"])<5 else np.mean(x["adj_analyst_bias"]))
analyst_avg_bias=analyst_avg_bias.to_frame(name="analyst_avg_bias").reset_index()
analyst_avg_bias["next_reporting_period"]=analyst_avg_bias["REPORTING_PERIOD"].apply(lambda x:str(int(x)+10000))
del analyst_avg_bias["REPORTING_PERIOD"]

df_est["ANN_PERIOD"]=df_est["DataDate"].apply(lambda x:x[0:4]+"1231" if x[4:6]>'04' else str(int(x[0:4])-1)+'1231')
df_est=pd.merge(df_est,analyst_avg_bias,left_on=["ANALYST_NAME","ANN_PERIOD"],right_on=["ANALYST_NAME","next_reporting_period"],how="left")

In [ ]:
#对过去3个月（过去三个月研报数小于5的个股使用过去6月的数据，对于过去六个月没有研报的个股填充nan值）的数据进行过滤，同一分析师对同一个股的多篇研报只取最后一篇
#并对过去三个月的一致预期数据按照时间和分析师偏差进行双重加权（时间越近权重越大，分析师偏差越小权重越大）

date_calendar=pd.date_range("{}-01-01".format(START_YEAR),"2020-03-10")
date_calendar=[str(i)[0:4]+str(i)[5:7]+str(i)[8:10] for i in date_calendar]

def Process_Single_StkPeriod(df):
    date_ls=list(df["DataDate"].drop_duplicates())
    df.set_index("DataDate",inplace=True)
    df["DataDate"]=df.index
    
    #按照分析师上一年预测偏差以及研报发布时间进行双重加权
    def con_est_calcu(est_data):
        if len(est_data) ==0:
            return np.nan
        analyst_bias=est_data["analyst_avg_bias"]
        max_bias=np.nanmax(analyst_bias)
        min_bias=np.nanmin(analyst_bias)
        if max_bias==min_bias or (math.isnan(min_bias) or math.isnan(max_bias)):
            num = len(analyst_bias)
            weight_analyst=np.array([1/num]*num)
        else:    
            weight_analyst=analyst_bias.apply(lambda x:1 if math.isnan(x) else 3-2*(x-min_bias)/(max_bias-min_bias))
        weight_analyst=weight_analyst/np.sum(weight_analyst)

        est_data["date_to_now"]=est_data["DataDate"].apply(lambda x:date_calendar.index(date)-date_calendar.index(x))
        weight_date=np.power(0.5**(1/45),np.array(est_data["date_to_now"]))
        weight_date=weight_date/np.sum(weight_date)
        
        total_weight=weight_analyst*weight_date
        total_weight=np.array(total_weight/np.sum(total_weight))
        
        est_eps=est_data["EST_EPS_DILUTED"].values
        median = np.median(est_eps)
        MAD =  1.483 * np.median(np.abs(est_eps - median))
        est_eps = np.clip(est_eps,median-3*MAD,median+3*MAD)
        
        if len(est_eps)>=3:
            max_index = np.argmax(est_eps)
            min_index = np.argmin(est_eps)
            est_eps = np.delete(est_eps,[max_index,min_index])
            total_weight = np.delete(total_weight,[max_index,min_index])
            total_weight = total_weight/np.sum(total_weight)
            
        if len(est_eps)>=5:
            max_index = np.argmax(est_eps)
            min_index = np.argmin(est_eps)
            est_eps = np.delete(est_eps,[max_index,min_index])
            total_weight = np.delete(total_weight,[max_index,min_index])
            total_weight = total_weight/np.sum(total_weight)
        
        con_est_eps=np.sum(np.array(est_eps)*total_weight)

        return con_est_eps
    
    single_stkperiod_result=pd.DataFrame(index=date_ls,columns=["con_est_eps"])
    for date in date_ls:
        date_range=date_calendar[date_calendar.index(date)-89:date_calendar.index(date)+1]#去过去90天数据（包含今天）
        est_data=df.loc[date_range]
        est_data=est_data[pd.notnull(est_data["EST_EPS_DILUTED"])]
        est_data=est_data.groupby("ANALYST_NAME").apply(lambda x:x.iloc[-1])#同一分析师多篇研报只取最后一篇
        
        #如果过去90天研报数小于10，则取过去180天研报
        if len(est_data)<10:
            date_range=date_calendar[date_calendar.index(date)-179:date_calendar.index(date)+1]
            est_data=df.loc[date_range]
            est_data=est_data[pd.notnull(est_data["EST_EPS_DILUTED"])]
            
            """if (1<=len(est_data)<=3):
                con_est_data = est_data["EST_EPS_DILUTED"].iloc[-1]
                single_stkperiod_result.loc[date,"con_est_eps"]=con_est_data
                return single_stkperiod_result"""
            
        con_est_data=con_est_calcu(est_data)
        single_stkperiod_result.loc[date,"con_est_eps"]=con_est_data

    return single_stkperiod_result

In [ ]:
temp_df = df_est[(df_est["sid"]=='600380.SH') & (df_est["REPORTING_PERIOD"] == '20181231')]
temp_df=temp_df[temp_df["DataDate"]>'20171110']
df = Process_Single_StkPeriod(temp_df)
plt.plot(pd.to_datetime(df.index),np.array(df["con_est_eps"]))

In [ ]:
df_est.groupby("sid").apply(lambda x:x.to_csv(r"/share/intern_share/analyst_est_data/analyst_est_data_{}.csv"\
                           .format(x["sid"].iloc[0]),index=False))

In [ ]:
grouped=df_est.groupby(["sid","REPORTING_PERIOD"])
con_est_eps_df=apply_parallel(grouped,Process_Single_StkPeriod)
con_est_eps_df=con_est_eps_df.reset_index().rename(columns={"level_0":"sid","level_1":"REPORTING_PERIOD","level_2":"DataDate"})

#按照交易日来reindex
trade_calendar=get_trade_dates("{}0101".format(START_YEAR+1),"20200310")
trade_calendar=[str(i) for i in trade_calendar]

con_est_eps_df=con_est_eps_df.groupby(["sid","REPORTING_PERIOD"],as_index=False)\
                    .apply(lambda x:x.replace(0,np.nan).fillna(method="ffill"))
con_est_eps_df=con_est_eps_df.groupby(["sid","REPORTING_PERIOD"],as_index=False)\
                    .apply(lambda x:x.drop_duplicates("DataDate").set_index("DataDate").reindex(trade_calendar,method="ffill").dropna().reset_index())

con_est_eps_df.to_csv(r"result/con_eps_my_v3.csv",index=False)

In [ ]:
#取企业快报数据
sql_1 = """SELECT S_INFO_WINDCODE, ANN_DT, REPORT_PERIOD,NET_PROFIT_EXCL_MIN_INT_INC
from ASHAREPROFITEXPRESS where ANN_DT between '{}0101' and '20201231'""".format(START_YEAR)

df_ProfitExpress = pd.read_sql_query(sql_1, conn)
df_ProfitExpress.rename(columns={"S_INFO_WINDCODE":"sid","REPORT_PERIOD":"REPORTING_PERIOD"},inplace=True)

#取企业预报数据
sql_2 = """SELECT S_INFO_WINDCODE, S_PROFITNOTICE_DATE, S_PROFITNOTICE_PERIOD,S_PROFITNOTICE_NETPROFITMIN,S_PROFITNOTICE_NETPROFITMAX
from ASHAREPROFITNOTICE where S_PROFITNOTICE_DATE between '{}0101' and '20201231'""".format(START_YEAR)

df_ProfitNotice = pd.read_sql_query(sql_2, conn)
df_ProfitNotice.rename(columns={"S_INFO_WINDCODE":"sid","S_PROFITNOTICE_PERIOD":"REPORTING_PERIOD"},inplace=True)


In [ ]:
##如果企业发布了业绩预报或者快报，则直接采取业绩预报或者快报的数据
df_FA_EPS.set_index(["sid","REPORTING_PERIOD"],inplace=True)

def Fill_with_Notice(df):
    ann_date=df["S_PROFITNOTICE_DATE"][df.index[0]]
    sid=df["sid"][df.index[0]]
    totshare = df_now_tot_share.set_index('sid').loc[sid,'TOT_SHR']
    min_profit=df["S_PROFITNOTICE_NETPROFITMIN"][df.index[0]]
    max_profit=df["S_PROFITNOTICE_NETPROFITMAX"][df.index[0]]
    notice_eps=(min_profit+max_profit)/2/totshare
    try:
        df["con_est_eps"][df["DataDate"]>=date_calendar[date_calendar.index(ann_date)-1]]=notice_eps
    except ValueError as e:
        pass
    return df
    
con_est_eps_df_merged=pd.merge(con_est_eps_df,df_ProfitNotice,on=["sid","REPORTING_PERIOD"],how="left")
con_est_eps_df_merged=con_est_eps_df_merged.groupby(["sid","REPORTING_PERIOD"]).apply(Fill_with_Notice)


def Fill_with_Express(df):
    sid=df["sid"][df.index[0]]
    ann_date=df["ANN_DT"][df.index[0]]
    totshare = df_now_tot_share.set_index('sid').loc[sid,'TOT_SHR']
    express_eps=df["NET_PROFIT_EXCL_MIN_INT_INC"][df.index[0]]/totshare/10000
    try:
        df["con_est_eps"][df["DataDate"]>=date_calendar[date_calendar.index(ann_date)-1]]=express_eps
    except ValueError as e:
        pass
    return df
    
con_est_eps_df_merged=pd.merge(con_est_eps_df_merged,df_ProfitExpress,on=["sid","REPORTING_PERIOD"],how="left")
con_est_eps_df_merged=con_est_eps_df_merged.groupby(["sid","REPORTING_PERIOD"]).apply(Fill_with_Express)
con_est_eps_df_merged=con_est_eps_df_merged[["sid","REPORTING_PERIOD","DataDate","con_est_eps"]]

#在企业发布正式年报之后，使用正式年报数据代替一致预期值
def Fill_with_FA(df):
    sid=df["sid"][df.index[0]]
    ann_date=df["ANN_DT"][df.index[0]]
    totshare = df_now_tot_share.set_index('sid').loc[sid,'TOT_SHR']
    FA_eps=(df['S_FA_DEDUCTEDPROFIT']+df['S_FA_EXTRAORDINARY'])[df.index[0]]/totshare/10000
    try:
        df["con_est_eps"][df["DataDate"]>=date_calendar[date_calendar.index(ann_date)-1]]=FA_eps
    except ValueError as e:
        pass
    return df

df_FA_EPS.reset_index(inplace=True)
con_est_eps_df_merged=pd.merge(con_est_eps_df_merged,df_FA_EPS,on=["sid","REPORTING_PERIOD"],how="left")
con_est_eps_df_merged=con_est_eps_df_merged.groupby(["sid","REPORTING_PERIOD"]).apply(Fill_with_FA)

#储存数据
con_est_eps_df_merged=con_est_eps_df_merged.groupby(["sid","REPORTING_PERIOD"],as_index=False)\
                    .apply(lambda x:x.replace(0,np.nan).fillna(method="ffill"))
con_est_eps_df_merged=con_est_eps_df_merged[["sid","REPORTING_PERIOD","DataDate","con_est_eps"]]
con_est_eps_df_merged.to_csv(r"result/con_eps_my_v3.csv",index=False)

In [ ]:
#提取出hs300的数据
def get_hs300_data(df):
    date=df["DataDate"][df.index[0]]
    w_hs300=query_table("DailyBar",start_date=date,end_date=date,fields=["w_hs300","w_cs500"])
    hs300_stk_ls=w_hs300["sid"][w_hs300["w_hs300"]!=0].values
    df=df.set_index("sid").loc[hs300_stk_ls].reset_index()
    return df

grouped=con_est_eps_df_merged.groupby("DataDate")
hs300_con_est_eps=apply_parallel(grouped,get_hs300_data)
hs300_con_est_eps=hs300_con_est_eps.reset_index(drop=True)

In [ ]:
'''
#筛选出ROE、净利润增速、销售收入增速稳健、市值较大的股票
#size:前10%--> ROE：前35%--> growth：前35%

df_Barra_Factor=pd.read_csv(r"/home/ywang/proj_1/data/Asset_Exposure_Data_Calculated.csv",dtype={'DataDate':str})
df_Barra_Factor=pd.merge(df_Barra_Factor,indus_size_df,on=["sid","DataDate"],how='left')

sql="""select S_INFO_WINDCODE ,S_INFO_NAME,S_INFO_COMPNAME,S_INFO_LISTBOARD FROM ASHAREDESCRIPTION """
descrip_df=pd.read_sql(sql,conn)
descrip_df.rename(columns={'S_INFO_WINDCODE':'sid'},inplace=True)
df_Barra_Factor=pd.merge(df_Barra_Factor,descrip_df[['sid','S_INFO_COMPNAME']],on='sid')

def Stock_Filter(df):
    df=df.sort_values("mktcap",ascending=False).iloc[0:15]
    
    df["DataDate"]=df["DataDate"].apply(str)
    return df[["sid","DataDate","S_INFO_COMPNAME","BarraETOP","BarraSGRO","BarraEGRO","BarraEGIBS","BarraEGIBS_s"]]

grouped=df_Barra_Factor.groupby(["DataDate","L1_INDUSTRY"])
df_Stock_Pool=apply_parallel(grouped,Stock_Filter)
StockPool_con_est_eps=pd.merge(con_est_eps_df_final,df_Stock_Pool,on=["sid","DataDate"])
'''

In [3]:
#加入过去两年PE_TTM的 rolling_median 值（相较于mean值更为稳定，不易受极端值影响）

sql = """SELECT S_INFO_WINDCODE, TRADE_DT, S_VAL_PE_TTM,S_DQ_MV,S_VAL_PB_NEW from ASHAREEODDERIVATIVEINDICATOR
where TRADE_DT between '{}0101' and '20201231'""".format(START_YEAR-1)  
df_PETTM = pd.read_sql_query(sql, conn)
df_PETTM=df_PETTM.rename(columns={"S_INFO_WINDCODE":"sid","TRADE_DT":"DataDate"})
df_PETTM=df_PETTM.sort_values(["sid","DataDate"])
df_PETTM.to_csv(r"data/PETTM.csv",index=False)
'''
df_PETTM=pd.read_csv(r"data/PETTM.csv",dtype={"DataDate":str})

def Rolling_Median(df):
    df["pe_2y_rollingmedian"]=df["S_VAL_PE"].rolling(252*2).apply(np.median)
    return df

grouped=df_PETTM.groupby("sid")
df_PETTM_2y_rollingmedian=apply_parallel(grouped,Rolling_Median)
hs300_con_est_eps=pd.merge(hs300_con_est_eps,df_PETTM_2y_rollingmedian,on=["sid","DataDate"],how="left")
'''

'\ndf_PETTM=pd.read_csv(r"data/PETTM.csv",dtype={"DataDate":str})\n\ndef Rolling_Median(df):\n    df["pe_2y_rollingmedian"]=df["S_VAL_PE"].rolling(252*2).apply(np.median)\n    return df\n\ngrouped=df_PETTM.groupby("sid")\ndf_PETTM_2y_rollingmedian=apply_parallel(grouped,Rolling_Median)\nhs300_con_est_eps=pd.merge(hs300_con_est_eps,df_PETTM_2y_rollingmedian,on=["sid","DataDate"],how="left")\n'

In [ ]:
#加入每天的收盘价序列

sql = """SELECT S_INFO_WINDCODE, TRADE_DT, S_DQ_ADJCLOSE,S_DQ_ADJFACTOR  from ASHAREEODPRICES 
where TRADE_DT between '{}0101' and '20201231'""".format(START_YEAR+1)    
df_close_hfq = pd.read_sql_query(sql, conn)
df_close_hfq=df_close_hfq.rename(columns={"S_INFO_WINDCODE":"sid","TRADE_DT":"DataDate"})
df_close_hfq=df_close_hfq.sort_values(["sid","DataDate"])
def fun(df):
    df["S_DQ_ADJCLOSE"]=df["S_DQ_ADJCLOSE"]/df["S_DQ_ADJFACTOR"].iloc[-1]
    return df
df_close_qfq=df_close_hfq.groupby("sid").apply(fun)
df_close_qfq.to_csv(r"data/close_price.csv",index=False)
'''
df_close=pd.read_csv(r"data/close_price.csv",dtype={"DataDate":str})

hs300_con_est_eps=pd.merge(hs300_con_est_eps,df_close,on=["sid","DataDate"],how="left")
'''